In [ ]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import calendar
import re
from datetime import datetime
import app1
import app2

app = dash.Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])

app.config['suppress_callback_exceptions'] = True

global df


tabs_styles = {
    'height': '44px',
    'align-items': 'center',
    'margin':'10px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold',
    'border-radius': '15px',
    'background-color': '#F2F2F2',
    'box-shadow': '4px 4px 4px 4px lightgrey',
    'margin':'10px'

}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#119DFF',
    'color': 'white',
    'padding': '6px',
    'border-radius': '15px',
}



title = html.Div([
          html.H1(['Dashboard for Covid-19 Statistics'],style={'text-align':'center'}),
          html.Hr()
          ])

app.layout = html.Div([

    title,
    dcc.Tabs(id="tabs-example", value='tab-1-example', children=[
        dcc.Tab(label='Covid Analysis', value='tab-1-example',style = tab_style, selected_style = tab_selected_style),
        dcc.Tab(label='Information', value='tab-2-example',style = tab_style, selected_style = tab_selected_style),
    ],style = tabs_styles),
    dbc.Container(id="tabs-content-example",fluid=True)
])

@app.callback(Output('tabs-content-example', 'children'),
              [Input('tabs-example', 'value')])
def render_content(tab):
    global df
    if tab == 'tab-1-example':
        df = app1.df1

        return app1.tab_1_layout
    elif tab == 'tab-2-example':
        df = app2.df2

        return app2.tab_2_layout



@app.callback(
              [Output('submit-button','n_clicks'),Output('dd1','value'),Output('dd2','value'),Output('date-range','start_date'),Output('date-range','end_date')],
              [Input('clear-button','n_clicks')],
              [State('submit-button','n_clicks'),
               State('dd1','value'),
               State('dd2','value'),
               State('date-range','start_date'),
               State('date-range','end_date')],
               prevent_initial_call=True

)

def clear_values(n_clicks_clear,n_clicks_submit,options1,options2,start_date,end_date):
    if n_clicks_clear > 0:
        n_clicks_submit= 0
        options1 = []
        options2 = []
        start_date = ""
        end_date = ""
        return [n_clicks_submit,options1,options2,start_date,end_date]
    else:
        return [n_clicks_submit,options1,options2,start_date,end_date]



@app.callback(
              [Output("modal-centered", "is_open"),Output("modal-body","children"),Output("visit-link","href")],
              [Input('plot','clickData'), Input("close-centered", "n_clicks")],
              [State("modal-centered", "is_open")]
)
def click_data(data,close,is_open):
    if data or close:
        list1 = data['points']
        list_customdata = list1[0]['customdata']
        link = list_customdata[1]
        if link is not None:
            return not is_open,link,link
    return is_open,""


@app.callback(
             [Output('plot','figure'),Output('loading-output','style'),Output('links','children')],
              [Input('submit-button','n_clicks')],
              [State('dd1','value'),
               State('dd2','value'),
               # State('slider','value'),
               State('date-range','start_date'),
               State('date-range','end_date')],
               prevent_initial_call=True
    )

def update_graph(n_clicks,dd1_value,dd2_value,start_date,end_date):
    if n_clicks >0 :
        start= datetime.strptime(start_date[:10],'%Y-%m-%d')
        end= datetime.strptime(end_date[:10],'%Y-%m-%d')

        fig = go.Figure()

        if(dd1_value is not None):
            for x in dd1_value:
                dd2_value.append(x)
        
        frames = []

        for tic in dd2_value:
            style_loading = {'visibility':'visible'}

            df_scatter = df[(df.apply(lambda r: r.str.contains(tic, case=False).any(), axis=1)) & (df['Date'] >= start) & (df['Date'] <= end)]
            

            fig.add_trace(go.Scatter({'x':df_scatter['Date'],
                                      'y':df_scatter['Time'],
                                      'name':tic,
                                      'mode':'markers',
                                      'marker':{'size':10,'line':{'color':'#000000','width':1}},
                                      'customdata':df_scatter.loc[:, ['Tweet','Link']],
                                      'hovertemplate':"Tweet: %{customdata[0]}<br>" +"Link: %{customdata[1]}<br>"+ "<extra></extra>"
                                      }))
        fig.update_layout(title='Covid-19 Twitter Scatter Plot',hoverlabel=dict(
        font_size=16,
        font_family="Rockwell",
        namelength = -1)
        )
        fig.update_yaxes(categoryorder='category ascending')

        for tic in dd2_value:

            frames.append(df[(df.apply(lambda r: r.str.contains(tic, case=False).any(), axis=1)) & (df['Date'] >= start_date) & (df['Date'] <= end_date)])

        df_filtered = pd.concat(frames,ignore_index=True)

        t_links =  df_filtered['Link'].tolist()

        t_links_final = [x for x in t_links if str(x) != 'nan']

        for i in t_links_final:
            list_of_links = html.Ul(id='links_ul',children=[html.Li(id=i,children=[html.A(href=i,children=i,target="_blank")]) for i in t_links_final])


        return fig,style_loading,list_of_links



if __name__ == "__main__":
    app.run_server(debug=False,dev_tools_ui=False,dev_tools_props_check=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Aug/2021 11:28:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 11:28:03] "GET /assets/style.css?m=1628886160.0 HTTP/1.1" 304 -
127.0.0.1 - - [17/Aug/2021 11:28:03] "GET /_dash-component-suites/dash/deps/polyfill@7.v1_21_0m1627155712.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 11:28:03] "GET /_dash-component-suites/dash/deps/react-dom@16.v1_21_0m1627155712.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 11:28:03] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v1_21_0m1627155712.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 11:28:03] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v0_13_0m1627999188.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 11:28:03] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_17_1m1627155711.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 11:28:03] "GET /_dash-layou

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/Users/ranjan/opt/ana

127.0.0.1 - - [17/Aug/2021 11:28:04] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Aug/2021 11:28:09] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/Users/ranjan/opt/ana

...

127.0.0.1 - - [17/Aug/2021 11:31:42] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Aug/2021 11:31:46] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/Users/ranjan/opt/ana

127.0.0.1 - - [17/Aug/2021 11:31:46] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Aug/2021 11:31:53] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/ranjan/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/Users/ranjan/opt/ana

127.0.0.1 - - [17/Aug/2021 11:31:53] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Aug/2021 11:41:32] "POST /_dash-update-component HTTP/1.1" 200 -
